# Optimización básica para cuenca piloto



Cuatro cuencas: dos de cabecera, una intermedia y una final

**C1**: Cuenca de cabecera
*   Pluviómetro: *SPE00156270*
*   Embalse *E065: Baserca* VOL máximo: 21.793Hm³

**C2**: Cuenca de cabecera
*   Pluviómetro: *SPE0015801*
*   Embalse *E063: Cavallers* VOL máximo: 16.046Hm³

**C3**: Cuenca intermedia
*   Pluviómetro: *SPE0015153*
*   Embalse *E050: Escales* VOL máximo: 152.317Hm³
*   Aforo: *A137*; Umbrales[ Info:120 ]

**C4**: Cuenca de cierre
*   Aforo: *A115*; Umbrales[ Info:130, PreAlarma: 210 ]

### Imports básicos

In [1]:
pip install pymoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210078 sha256=917e2bbe87b37ac0aac2ae5c5b3d50c00eb9709649fe61f5da710ff783d41e0e
  Stored in directory: /root/.cache/pip/wheels/01/e1/49/37e6bde9886439057450c494a79b0bef8bbe897a54aebfc757
Successfully built grapheme


In [2]:
from pymoo.core.problem import ElementwiseProblem
from pymoo.optimize import minimize
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.sampling.rnd import FloatRandomSampling

### Problema optimización dos cuencas en serie

In [9]:
from pymoo.core.problem import ElementwiseProblem
from pymoo.optimize import minimize
from pymoo.algorithms.soo.nonconvex.ga import GA
import numpy as np

class DosEmbalsesEnSerie(ElementwiseProblem):

    def __init__(self, inflow1, inflow2, current_storage1, current_storage2, max_capacity1, max_capacity2, threshold1, threshold2):
        self.inflow1 = inflow1
        self.inflow2 = inflow2
        self.current_storage1 = current_storage1
        self.current_storage2 = current_storage2
        self.max_capacity1 = max_capacity1
        self.max_capacity2 = max_capacity2
        self.threshold1 = threshold1
        self.threshold2 = threshold2

        # Define lower and upper bounds for the storage variables
        xl = np.array([0, 0])  # Minimum storage is 0
        xu = np.array([max_capacity1, max_capacity2])  # Maximum storage is up to capacity

        super().__init__(n_var=2, n_obj=3, n_constr=6, xl=xl, xu=xu)

    def _evaluate(self, x, out, *args, **kwargs):
        storage_goal1 = x[0]
        storage_goal2 = x[1]

        # Cuenca 1
        storage1 = np.clip(self.current_storage1 + self.inflow1 - storage_goal1, 0, self.max_capacity1)
        released_water1 = self.inflow1 + self.current_storage1 - storage_goal1
        excess_release1 = max(0, released_water1 - self.threshold1)

        # Cuenca 2
        inflow2_adjusted = self.inflow2 + released_water1
        storage2 = np.clip(self.current_storage2 + inflow2_adjusted - storage_goal2, 0, self.max_capacity2)
        released_water2 = inflow2_adjusted + self.current_storage2 - storage_goal2
        excess_release2 = max(0, released_water2 - self.threshold2)

        # La función objetivo es doble:
        # 1. Maximizar el agua almacenada en ambos embalses (convertido a minimización)
        # 2. Minimizar el exceso de agua liberada sobre los umbrales
        #out["F"] = np.array([-storage1 - storage2, excess_release1 + excess_release2])

        f1 = storage1 + storage2  # Maximizar agua embalsada
        f2 = -excess_release1
        f3 = -excess_release2
        out["F"] = np.column_stack([f1, f2, f3]) # Stack objectives as columns

        # Restricciones
        g1 = -storage1  # El agua almacenada en el embalse 1 no puede ser negativa
        g2 = storage1 - self.max_capacity1  # El agua almacenada en el embalse 1 no puede superar la capacidad máxima
        g3 = -storage2  # El agua almacenada en el embalse 2 no puede ser negativa
        g4 = storage2 - self.max_capacity2  # El agua almacenada en el embalse 2 no puede superar la capacidad máxima
        g5 = -excess_release1  # El agua soltada desde la cuenca 1 no puede ser negativa
        g6 = -excess_release2  # El agua soltada desde la cuenca 2 no puede ser negativa

        out["G"] = np.array([g1, g2, g3, g4, g5, g6])

# Ejemplo de datos
inflow1 = 60              # Agua entrante en el ciclo actual para la cuenca 1
current_storage1 = 20     # Estado actual del embalse 1
max_capacity1 = 50        # Capacidad máxima del embalse 1
threshold1 = 20           # Umbral de seguridad para el caudal de agua en la cuenca 1

inflow2 = 20               # Agua entrante en el ciclo actual para la cuenca 2
current_storage2 = 10     # Estado actual del embalse 2
max_capacity2 = 30        # Capacidad máxima del embalse 2
threshold2 = 8            # Umbral de seguridad para el caudal de agua en la cuenca 2

problem = DosEmbalsesEnSerie(
    inflow1=inflow1,
    inflow2=inflow2,
    current_storage1=current_storage1,
    current_storage2=current_storage2,
    max_capacity1=max_capacity1,
    max_capacity2=max_capacity2,
    threshold1=threshold1,
    threshold2=threshold2
)

#algorithm = GA(pop_size=20)
algorithm = NSGA2(pop_size=20)


res = minimize(problem,
               algorithm,
               termination=('n_gen', 50),
               seed=1,
               verbose=True)

print("Mejor solución encontrada: \nX = %s\nF = %s" % (res.X, res.F))


n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |       20 |      7 |  0.000000E+00 |  0.000000E+00 |             - |             -
     2 |       40 |      9 |  0.000000E+00 |  0.000000E+00 |  0.0151097556 |         ideal
     3 |       60 |     13 |  0.000000E+00 |  0.000000E+00 |  0.0360030689 |             f
     4 |       80 |     20 |  0.000000E+00 |  0.000000E+00 |  0.0307957709 |         ideal
     5 |      100 |     20 |  0.000000E+00 |  0.000000E+00 |  0.0312887146 |         ideal
     6 |      120 |     20 |  0.000000E+00 |  0.000000E+00 |  0.0710096080 |         nadir
     7 |      140 |     20 |  0.000000E+00 |  0.000000E+00 |  0.0133498476 |         ideal
     8 |      160 |     20 |  0.000000E+00 |  0.000000E+00 |  0.0085467292 |             f
     9 |      180 |     20 |  0.000000E+00 |  0.000000E+00 |  0.0129591930 |         ideal
    10 |      200 |     20 |  0.000000E+00 |  0.000000E+00 |  0.0319867966 |         ideal

### Optimización cuenca piloto

In [13]:
from pymoo.core.problem import ElementwiseProblem
from pymoo.optimize import minimize
from pymoo.algorithms.soo.nonconvex.ga import GA
import numpy as np

class CuencaPilotoProblem(ElementwiseProblem):

    def __init__(self, inflow1, inflow2, inflow3
                 , current_storage1, current_storage2, current_storage3
                 , max_capacity1, max_capacity2, max_capacity3
                 , threshold3, threshold4):
        self.inflow1 = inflow1
        self.inflow2 = inflow2
        self.inflow3 = inflow3
        self.current_storage1 = current_storage1
        self.current_storage2 = current_storage2
        self.current_storage3 = current_storage3
        self.max_capacity1 = max_capacity1
        self.max_capacity2 = max_capacity2
        self.max_capacity3 = max_capacity3
        self.threshold3 = threshold3
        self.threshold4 = threshold4

        # Define lower and upper bounds for the storage variables
        xl = np.array([0, 0, 0])  # Minimum storage is 0
        xu = np.array([max_capacity1, max_capacity2,  max_capacity3])  # Maximum storage is up to capacity

        super().__init__(n_var=3, n_obj=3, n_constr=8, xl=xl, xu=xu)

    def _evaluate(self, x, out, *args, **kwargs):
        storage_goal1 = x[0]
        storage_goal2 = x[1]
        storage_goal3 = x[2]

        # C1
        storage1 = np.clip(self.current_storage1 + self.inflow1 - storage_goal1, 0, self.max_capacity1)
        released_water1 = self.inflow1 + self.current_storage1 - storage_goal1

        # C2
        storage2 = np.clip(self.current_storage2 + self.inflow2 - storage_goal2, 0, self.max_capacity2)
        released_water2 = self.inflow2 + self.current_storage2 - storage_goal2

        # C3
        inflow3_adjusted = self.inflow3 + released_water1 + released_water2
        storage3 = np.clip(self.current_storage3 + inflow3_adjusted - storage_goal3, 0, self.max_capacity3)
        released_water3 = inflow3_adjusted + self.current_storage3 - storage_goal3
        excess_release3 = max(0, released_water3 - self.threshold3)

        # C4
        inflow4_adjusted = released_water3
        released_water4 = inflow4_adjusted
        excess_release4 = max(0, released_water4 - self.threshold4)

        # La función objetivo es doble:
        # 1. Maximizar el agua almacenada en ambos embalses (convertido a minimización)
        # 2. Minimizar el exceso de agua liberada sobre los umbrales controlados en aforos

        f_storage = storage1 + storage2 + storage3  # Maximizar agua embalsada
        f_a137 = abs( self.threshold3 - excess_release3)
        f_a115 = abs( self.threshold4 - excess_release4)
        out["F"] = np.column_stack([f_storage, f_a137, f_a115]) # Stack objectives as columns

        # Restricciones
        g1 = -storage1  # El agua almacenada en el embalse E065 no puede ser negativa
        g2 = storage1 - self.max_capacity1  # El agua almacenada en el embalse E065 no puede superar la capacidad máxima

        g3 = -storage2  # El agua almacenada en el embalse E063 no puede ser negativa
        g4 = storage2 - self.max_capacity2  # El agua almacenada en el embalse E063 no puede superar la capacidad máxima

        g5 = -storage3  # El agua almacenada en el embalse E050 no puede ser negativa
        g6 = storage3 - self.max_capacity3  # El agua almacenada en el embalse E050 no puede superar la capacidad máxima

        g7 = -excess_release3  # El agua que pasa por A137 no puede ser negativa
        g8 = -excess_release4  # El agua que pasa por A15 no puede ser negativa

        out["G"] = np.array([g1, g2, g3, g4, g5, g6, g7, g8])

# Ejemplo de datos
#C1
inflow1 = 60              # Agua entrante en el ciclo actual en el pluvio SPE00156270
current_storage1 = 20     # Estado actual del embalse E065
max_capacity1 = 50        # Capacidad máxima del embalse E065

#C2
inflow2 = 10              # Agua entrante en el ciclo actual en el pluvio SPE00156801
current_storage2 = 20     # Estado actual del embalse E063
max_capacity2 = 50        # Capacidad máxima del embalse E063


#C3
inflow3 = 20               # Agua entrante en el ciclo actual en el pluvio SPE00156153
current_storage3 = 10     # Estado actual del embalse 2
max_capacity3 = 30        # Capacidad máxima del embalse 2
threshold3 = 8            # Umbral de seguridad para el caudal de agua en el agoro A137

#C4
threshold4 = 18            # Umbral de seguridad para el caudal de agua en el agoro A115


problem = CuencaPilotoProblem(
    inflow1=inflow1,
    inflow2=inflow2,
    inflow3=inflow3,
    current_storage1=current_storage1,
    current_storage2=current_storage2,
    current_storage3=current_storage3,
    max_capacity1=max_capacity1,
    max_capacity2=max_capacity2,
    max_capacity3=max_capacity3,
    threshold3=threshold3,
    threshold4=threshold4
)

algorithm = NSGA2(pop_size=20)


res = minimize(problem,
               algorithm,
               termination=('n_gen', 50),
               seed=1,
               verbose=True)

print("Mejor solución encontrada: \nX = %s\nF = %s" % (res.X, res.F))


n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |       20 |      2 |  0.000000E+00 |  0.000000E+00 |             - |             -
     2 |       40 |      5 |  0.000000E+00 |  0.000000E+00 |  0.5698825488 |         ideal
     3 |       60 |     10 |  0.000000E+00 |  0.000000E+00 |  0.2326591238 |         ideal
     4 |       80 |     20 |  0.000000E+00 |  0.000000E+00 |  0.0401206252 |         ideal
     5 |      100 |     20 |  0.000000E+00 |  0.000000E+00 |  0.0976618550 |         ideal
     6 |      120 |     20 |  0.000000E+00 |  0.000000E+00 |  0.1522081916 |         ideal
     7 |      140 |     20 |  0.000000E+00 |  0.000000E+00 |  0.0065220542 |         ideal
     8 |      160 |     20 |  0.000000E+00 |  0.000000E+00 |  0.0259683312 |         ideal
     9 |      180 |     20 |  0.000000E+00 |  0.000000E+00 |  0.0186882933 |         ideal
    10 |      200 |     20 |  0.000000E+00 |  0.000000E+00 |  0.0134453826 |         ideal